In [50]:
!pip install --upgrade pip
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker --quiet

In [51]:
%pip install --upgrade transformers==4.27.2 --quiet
%pip install --upgrade datasets==2.11.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [52]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

aws_role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

In [53]:
model_id, model_version = "huggingface-textgeneration1-gpt-j-6b", "*"

In [56]:
output_bucket = sess.default_bucket()
output_prefix = "rs-domain-train"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

In [63]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

endpoint_name_after_finetune = name_from_base(f"rs-domaindata-{model_id}")

inference_instance_type = "ml.p3.8xlarge"

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the model uri.
#model_uri = "s3://sagemaker-us-east-1-588792541783/rs-domain-train/output/rs-huggingface-textgeneration1-gpt-j-6b-2023-08-04-00-56-41-499/output/model.tar.gz"
#model_uri = "s3://sagemaker-us-east-1-588792541783/rs-domain-train/output/rs-huggingface-textgeneration1-gpt-j-6b-2023-08-04-03-47-04-127/output/model.tar.gz"
model_uri = "s3://sagemaker-us-east-1-588792541783/rs-domain-train/output/rs-1-huggingface-textgeneration1-gpt-j--2023-08-04-09-37-07-153/output/model.tar.gz"


# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name_after_finetune,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py39.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.p3.8xlarge.


In [64]:

# deploy the Model. TODO
domain_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name_after_finetune,
)

INFO:sagemaker:Creating model with name: rs-domaindata-huggingface-textgeneratio-2023-08-04-10-42-31-363
INFO:sagemaker:Creating endpoint-config with name rs-domaindata-huggingface-textgeneratio-2023-08-04-10-42-31-363
INFO:sagemaker:Creating endpoint with name rs-domaindata-huggingface-textgeneratio-2023-08-04-10-42-31-363


-------------!

In [35]:
import json
import boto3


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


def parse_response_multiple_texts(query_response):
    generated_text = []
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]

In [65]:
parameters = {
    "max_length": 100,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 0.1,
}

res_gpt_finetune = []
for quota_text in [
    "Amazon CodeWhisperer is a",
    "Swanky Glasses is an item with",
    "Gainsboro Work Table is an item with id",
]:
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name_after_finetune
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_finetune.append(generated_texts)
    print(generated_texts)
    print("\n")

Amazon CodeWhisperer is a:software development tool, code review, code quality, code metrics, code auditing, code analysis, code refactoring, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation, code generation,


Swanky Glasses is an item with: glasses wedding ceremonial and is sold at a price of $55.99 and has 10 units available. Wedding-ceremonial glass is a must-have for your wedding Dark Slate Gray Jacket is an item with id f8a9e8a9-e9e3-4a5b-a8e7-f8e7f8e7f8e7 and style jacket


Gainsboro Work Table is an item with id: f9e7a8e7-f9e7-4b5e-a9e9-e0a9e9e8a9e5 and style tables from the furniture category. This item is sold at a price of $213.99 and has 10 units available. This gainsboro 

In [66]:
parameters = {
    "max_length": 100,
    "num_return_sequences": 1,
    "top_k": 25,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 0.5,
}

res_gpt_finetune = []
for quota_text in [
    "what is the price of swanky glass",
]:
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name_after_finetune
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_finetune.append(generated_texts)
    print(generated_texts)
    print("\n")

what is the price of swanky glass: an item with id d2b9f6e9-b5d7-4c7b-b8b8-d0e0a8b4e7b0 and style kitchen from the housewares category. This item is sold at a price of $64.99 and has 17 units available. Unequaled for every kitchen Dark Slate Gray Jacket is an item with id 6d7f2c




In [67]:
parameters = {
    "max_length": 100,
    "num_return_sequences": 1,
    "top_k": 10,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 0.2,
}

res_gpt_finetune = []
for quota_text in [
    "Define SageMaker in one sentence. Give the response in English and French languages",
]:
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name_after_finetune
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_finetune.append(generated_texts)
    print(generated_texts)
    print("\n")

Define SageMaker in one sentence. Give the response in English and French languages: SageMaker is a fully managed service for data science and ML that makes it easier to build, train, and deploy ML models.SageMaker is a fully managed service for data science and ML that makes it easier to build, train, and deploy ML models.SageMaker is a fully managed service for data science and ML that makes it easier to build, train, and deploy ML models.SageMaker




In [68]:
parameters = {
    "max_length": 100,
    "num_return_sequences": 1,
    "top_k": 10,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 0.2,
}

res_gpt_finetune = []
for quota_text in [
    "can you explain CodeWhisperer features?",
]:
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name_after_finetune
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_finetune.append(generated_texts)
    print(generated_texts)
    print("\n")

can you explain CodeWhisperer features?:

CodeWhisperer is a code reviewer that helps developers write better code by suggesting code reviews and code refactoring suggestions. CodeWhisperer uses a deep learning model to analyze code and generate suggestions to help you write better code. CodeWhisperer uses a deep learning model to analyze code and generate suggestions to help you write better code. CodeWhisperer uses a deep learning model to analyze code and generate suggestions


